In [1]:
import math
from collections import Counter

def frequency_test(bits):
    """
    Frequency Test (Monobit Test): Checks if the number of 0s and 1s in a bit string are approximately equal.

    Args:
        bits: A string of '0's and '1's representing the bit sequence.

    Returns:
        (p-value, bool): The p-value of the test and True if the sequence passes (p-value >= 0.01), False otherwise.
    """
    n = len(bits)
    s = abs(bits.count('1') - (n / 2))
    p_value = math.erfc(s / math.sqrt(n))
    return p_value, p_value >= 0.01

def runs_test(bits):
    """
    Runs Test: Checks if the number of runs (consecutive sequences of the same bit) is as expected for a random sequence.

    Args:
        bits: A string of '0's and '1's representing the bit sequence.

    Returns:
        (p-value, bool): The p-value of the test and True if the sequence passes (p-value >= 0.01), False otherwise.
    """
    n = len(bits)
    pi = bits.count('1') / n
    tau = 2 / math.sqrt(n)
    if abs(pi - 0.5) > tau:
        return 0.0, False
    
    runs = 1
    for i in range(1, n):
        if bits[i] != bits[i - 1]:
            runs += 1
    
    R_obs = runs
    mu = 2 * n * pi * (1 - pi)
    sigma_sq = 2 * pi * (1 - pi) * (2 * n * pi * (1 - pi) - n) / (n - 1)
    p_value = math.erfc(abs(R_obs - mu) / (math.sqrt(2 * sigma_sq)))
    return p_value, p_value >= 0.01

def longest_run_of_ones_test(bits):
    """
    Longest Run of Ones Test: Checks if the longest run of 1s in the sequence is within the expected range for a random sequence.

    Args:
        bits: A string of '0's and '1's representing the bit sequence.

    Returns:
        (p-value, bool): The p-value of the test and True if the sequence passes (p-value >= 0.01), False otherwise.
    """
    n = len(bits)
    if n < 128:
        return 0.0, False
    
    k, m = 3, 8  # For n <= 6272
    if n >= 6272:
        k, m = 5, 128
    
    nu = [0] * (k + 1)
    run_length = 0
    for bit in bits:
        if bit == '1':
            run_length += 1
        else:
            if run_length < m:
                nu[run_length] += 1
            else:
                nu[k] += 1
            run_length = 0
    if run_length < m:
        nu[run_length] += 1
    else:
        nu[k] += 1

    pi = [0.2148, 0.3672, 0.2305, 0.1875]  # For k = 3
    if k == 5:
        pi = [0.1174, 0.2430, 0.2493, 0.1753, 0.1027, 0.1124]
    
    chi_sq = sum(((nu[i] - (n * pi[i]))**2) / (n * pi[i]) for i in range(k))
    p_value = 1 - math.gamma(k / 2, chi_sq / 2) / math.gamma(k / 2)
    return p_value, p_value >= 0.01

def approximate_entropy_test(bits, m=10):
    """
    Approximate Entropy Test: Checks the randomness of blocks of consecutive bits in the sequence.

    Args:
        bits: A string of '0's and '1's representing the bit sequence.
        m: The length of blocks to consider.

    Returns:
        (p-value, bool): The p-value of the test and True if the sequence passes (p-value >= 0.01), False otherwise.
    """
    n = len(bits)
    if n < m:
        return 0.0, False
    
    def get_counts(seq):
        return dict(Counter([seq[i:i+m] for i in range(len(seq)-m+1)]))
    
    C_m = [val/float(n-m+1) for val in get_counts(bits).values()]
    C_m1 = [val/float(n-m) for val in get_counts(bits[:n-1]).values()]

    phi_m = sum([c * math.log(c) for c in C_m if c > 0])
    phi_m1 = sum([c * math.log(c) for c in C_m1 if c > 0])

    ApEn_m = phi_m1 - phi_m
    chi_sq = 2 * (n-m+1) * (math.log(2) - ApEn_m)
    p_value = 1 - math.gamma(2**(m-1), chi_sq/2.0)/math.gamma(2**(m-1))
    return p_value, p_value >= 0.01
 
 # Example usage (convert your random numbers to a bit string)
random_numbers = [0.2, 0.7, 0.1, 0.9, 0.3] # Example
bits = ''.join(['1' if n >= 0.5 else '0' for n in random_numbers])
print(f"Bits: {bits}")

tests = {
    "Frequency Test": frequency_test,
    "Runs Test": runs_test,
    "Longest Run of Ones Test": longest_run_of_ones_test,
    "Approximate Entropy Test": approximate_entropy_test
}

for test_name, test_func in tests.items():
    p_value, passed = test_func(bits)
    print(f"{test_name}: p-value = {p_value:.4f}, Passed = {passed}")

Bits: 01010
Frequency Test: p-value = 0.7518, Passed = True


ValueError: math domain error